In [ ]:
!git clone https://github.com/zyainfal/One-Shot-Face-Swapping-on-Megapixels.git
%cd /content/One-Shot-Face-Swapping-on-Megapixels/inference

!git clone https://github.com/rosinality/stylegan2-pytorch.git
!cp -r /content/One-Shot-Face-Swapping-on-Megapixels/inference/stylegan2-pytorch/* /content/One-Shot-Face-Swapping-on-Megapixels/inference/
!pip install ninja

Cloning into 'One-Shot-Face-Swapping-on-Megapixels'...
remote: Enumerating objects: 319, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 319 (delta 15), reused 13 (delta 11), pack-reused 299 (from 1)
Receiving objects: 100% (319/319), 11.42 MiB | 13.81 MiB/s, done.
Resolving deltas: 100% (108/108), done.
/content/One-Shot-Face-Swapping-on-Megapixels/inference
Cloning into 'stylegan2-pytorch'...
remote: Enumerating objects: 395, done.
remote: Total 395 (delta 0), reused 0 (delta 0), pack-reused 395 (from 1)
Receiving objects: 100% (395/395), 122.51 MiB | 21.16 MiB/s, done.
Resolving deltas: 100% (203/203), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 7.7 MB/s eta 0:00:00


In [ ]:
%cd /content
!git clone https://github.com/rosinality/stylegan2-pytorch.git
%cd /content/stylegan2-pytorch
!git clone https://github.com/NVlabs/stylegan2.git

!cp -r /content/stylegan2-pytorch/stylegan2/* /content/stylegan2-pytorch/

/content
Cloning into 'stylegan2-pytorch'...
remote: Enumerating objects: 395, done.
remote: Total 395 (delta 0), reused 0 (delta 0), pack-reused 395 (from 1)
Receiving objects: 100% (395/395), 122.51 MiB | 10.43 MiB/s, done.
Resolving deltas: 100% (205/205), done.
Updating files: 100% (49/49), done.
/content/stylegan2-pytorch
Cloning into 'stylegan2'...
remote: Enumerating objects: 138, done.
remote: Total 138 (delta 0), reused 0 (delta 0), pack-reused 138 (from 1)
Receiving objects: 100% (138/138), 594.91 KiB | 1.89 MiB/s, done.
Resolving deltas: 100% (64/64), done.


In [ ]:
from huggingface_hub import hf_hub_download
weight_path = hf_hub_download(repo_id="akhaliq/OneshotCLIP-stylegan2-ffhq", filename="stylegan2-ffhq-config-f.pt")

stylegan2-ffhq-config-f.pt:   0%|          | 0.00/381M [00:00<?, ?B/s]

In [ ]:
import os

cmd = f"!cp {weight_path} /content/One-Shot-Face-Swapping-on-Megapixels/inference/checkpoint/"
!${cmd}

os.rename("/content/One-Shot-Face-Swapping-on-Megapixels/inference/checkpoint/stylegan2-ffhq-config-f.pt",
          "/content/One-Shot-Face-Swapping-on-Megapixels/inference/checkpoint/stylegan2-ffhq-config-f.pth"
          )

!cp "/content/drive/MyDrive/Colab_Notebooks/CV/gasn/data/ftm_final.pth" /content/One-Shot-Face-Swapping-on-Megapixels/inference/checkpoint/

In [ ]:
%cd /content
# !gdown 1badu11NqxGf6qM3PTTooQDJvQbejgbTv
# !unzip /content/CelebAMask-HQ.zip

!unzip /content/drive/MyDrive/Colab_Notebooks/CV/gasn/data/CelebAMask-HQ.zip
!unzip /content/drive/MyDrive/Colab_Notebooks/CV/gasn/data/CelebA-HQ-masks.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: content/masks_output/03822_mask.png  
  inflating: content/masks_output/13055_mask.png  
  inflating: content/masks_output/01617_mask.png  
  inflating: content/masks_output/07319_mask.png  
  inflating: content/masks_output/01794_mask.png  
  inflating: content/masks_output/02894_mask.png  
  inflating: content/masks_output/21388_mask.png  
  inflating: content/masks_output/04597_mask.png  
  inflating: content/masks_output/13980_mask.png  
  inflating: content/masks_output/09637_mask.png  
  inflating: content/masks_output/08136_mask.png  
  inflating: content/masks_output/18742_mask.png  
  inflating: content/masks_output/20477_mask.png  
  inflating: content/masks_output/13514_mask.png  
  inflating: content/masks_output/01659_mask.png  
  inflating: content/masks_output/20120_mask.png  
  inflating: content/masks_output/06761_mask.png  
  inflating: content/masks_output/09181_mask.png  
  inflating: c

In [ ]:
# from google.colab import userdata
# import os

# os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
# os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

# !kaggle datasets download -d lamsimon/celebahq
# !unzip "/content/celebahq.zip"

In [ ]:
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt

%cd /content/One-Shot-Face-Swapping-on-Megapixels/inference
from inference import MegaFS


str2num = {
    # 0: 'background',
    1: 'skin',
    2: 'l_brow',
    3: 'r_brow',
    4: 'l_eye',
    5: 'r_eye',
    6: 'eye_g',
    7: 'l_ear',
    8: 'r_ear',
    # 9: 'ear_r',
    10: 'nose',
    11: 'mouth',
    12: 'u_lip',
    13: 'l_lip',
    14: 'neck',
    # 15: 'neck_l',
    # 16: 'cloth',
    17: 'hair',
    # 18: 'hat'
}


class My_MegaFS(MegaFS):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def read_pair(self, src_idx, tgt_idx):
        src_face = cv2.imread(os.path.join(self.img_root, "{}.jpg".format(src_idx)))
        tgt_face = cv2.imread(os.path.join(self.img_root, "{}.jpg".format(tgt_idx)))
        tgt_mask  = cv2.imread(os.path.join(self.mask_root, "{}_mask.png".format(tgt_idx)))

        src_face_rgb = src_face[:, :, ::-1]
        tgt_face_rgb = tgt_face[:, :, ::-1]
        # tgt_mask = self.encode_segmentation_rgb(tgt_idx)
        return src_face_rgb, tgt_face_rgb, tgt_mask


    def run(self, src_idx, tgt_idx, refine=False, save_path='/content'):
        src_face_rgb, tgt_face_rgb, tgt_mask = self.read_pair(src_idx, tgt_idx)
        source, target = self.preprocess(src_face_rgb, tgt_face_rgb)
        swapped_face = self.swap(source, target)
        swapped_face = self.postprocess(swapped_face, tgt_face_rgb, tgt_mask)

        result = np.hstack((src_face_rgb[:,:,::-1], tgt_face_rgb[:,:,::-1], swapped_face))

        if refine: # True give some bad results...
            swapped_tensor, _ = self.preprocess(swapped_face[:,:,::-1], swapped_face)
            refined_face = self.refine(swapped_tensor)
            refined_face = self.postprocess(refined_face, tgt_face_rgb, tgt_mask)
            result = np.hstack((result, refined_face))

        save_path = f"{save_path}/{self.swap_type}_{src_idx}_and_{tgt_idx}"
        print(save_path)
        cv2.imwrite("{}.jpg".format(save_path), result)

/content/One-Shot-Face-Swapping-on-Megapixels/inference


/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


In [ ]:
handler = My_MegaFS(swap_type="ftm",
                 img_root="/content/CelebAMask-HQ/CelebA-HQ-img",
                 mask_root="/content/content/masks_output"
            )
# they have different sizes: masked 512x512, imgs 1024x1024 (???)

load encoder & swapper: ./checkpoint/ftm_final.pth


/content/One-Shot-Face-Swapping-on-Megapixels/inference/inference.py:79: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpts = torch.load(ckpt_e, map_location=torch.device("

load generator: ./checkpoint/stylegan2-ffhq-config-f.pth


/content/One-Shot-Face-Swapping-on-Megapixels/inference/inference.py:88: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpts = torch.load(ckpt_f, map_location=torch.device("

In [ ]:
handler.run(src_idx="10705", tgt_idx="10705")  # src_face_rgb, tgt_face_rgb, swapped_face, refined_swapped_face (somewhy has a bad quality)

data: (1024, 1024, 3) (1024, 1024, 3)


/content/One-Shot-Face-Swapping-on-Megapixels/inference/megafs.py:270: UserWarning: `nn.functional.upsample` is deprecated. Use `nn.functional.interpolate` instead.
  p2 = self.output2(c2) + F.upsample(p3, scale_factor=2, mode='bilinear', align_corners=True) # N, 512, 16, 16
/content/One-Shot-Face-Swapping-on-Megapixels/inference/megafs.py:271: UserWarning: `nn.functional.upsample` is deprecated. Use `nn.functional.interpolate` instead.
  p1 = self.output1(c1) + F.upsample(p2, scale_factor=2, mode='bilinear', align_corners=True) # N, 512, 32, 32


error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [ ]:
from torch.utils.cpp_extension import load


# module_path = os.path.dirname(__file__)
fused = load(
    "fused",
    sources=[
        # os.path.join(module_path, "fused_bias_act.cpp"),
        "/content/One-Shot-Face-Swapping-on-Megapixels/inference/op/fused_bias_act.cpp",
        # os.path.join(module_path, "fused_bias_act_kernel.cu"),
        # "/content/One-Shot-Face-Swapping-on-Megapixels/inference/op/fused_bias_act_kernel.cu"
    ],
)

ImportError: /root/.cache/torch_extensions/py311_cu124/fused/fused_v1.so: undefined symbol: _Z17fused_bias_act_opRKN2at6TensorES2_S2_iiff

## optimized execution for model

In [ ]:
import os
import cv2
from tqdm.notebook import tqdm
import torch
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.functional as tF
import torch
import gc


class FaceSwapDataset(Dataset):
    def __init__(self, img_root, mask_root, pairs):
        self.img_root = img_root
        self.mask_root = mask_root
        self.pairs = pairs  # List of (src_idx, tgt_idx)

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        src_idx, tgt_idx = self.pairs[idx]

        src_face = cv2.imread(os.path.join(self.img_root, f"{src_idx}.jpg"))
        tgt_face = cv2.imread(os.path.join(self.img_root, f"{tgt_idx}.jpg"))

        src_face_rgb = cv2.cvtColor(src_face, cv2.COLOR_BGR2RGB)
        tgt_face_rgb = cv2.cvtColor(tgt_face, cv2.COLOR_BGR2RGB)

        tgt_idx = "0" * (5 - len(str(tgt_idx))) + str(tgt_idx)
        tgt_mask = cv2.imread(os.path.join(self.mask_root, f"{tgt_idx}_mask.png"))

        # https://discuss.pytorch.org/t/negative-strides-in-tensor-error/134287/2
        src_face_rgb = np.ascontiguousarray(src_face[:, :, ::-1])  # Copy ensures no negative strides
        tgt_face_rgb = np.ascontiguousarray(tgt_face[:, :, ::-1])
        tgt_mask = np.ascontiguousarray(tgt_mask)

        src_face_rgb_ = cv2.resize(src_face_rgb.copy(), (256, 256))
        tgt_face_rgb_ = cv2.resize(tgt_face_rgb.copy(), (256, 256))
        src = torch.from_numpy(src_face_rgb_.transpose((2, 0, 1))).float().mul_(1/255.0)
        tgt = torch.from_numpy(tgt_face_rgb_.transpose((2, 0, 1))).float().mul_(1/255.0)

        src = tF.normalize(src, (0.5, 0.5, 0.5), (0.5, 0.5, 0.5), False)
        tgt = tF.normalize(tgt, (0.5, 0.5, 0.5), (0.5, 0.5, 0.5), False)

        return src_idx, tgt_idx, src_face_rgb, tgt_face_rgb, tgt_mask, src, tgt


def process_batch(batch, model, save_path):
    src_idx, tgt_idx, src_face_rgb, tgt_face_rgb, tgt_mask, source, target = batch
    # source, target = model.preprocess(src_face_rgb, tgt_face_rgb)

    swapped_face = model.swap(source, target)
    swapped_face = model.postprocess(swapped_face, tgt_face_rgb.numpy(), ~tgt_mask.squeeze(0).numpy())

    # result = np.hstack((src_face_rgb.numpy()[:, :, ::-1], tgt_face_rgb.numpy()[:, :, ::-1], swapped_face))
    # print(f"{result.shape=}")
    save_file = f"{save_path}/{model.swap_type}_src_{src_idx[0]}__tgt_{tgt_idx[0]}.jpg"
    cv2.imwrite(save_file, swapped_face[0][:, :, ::-1])


def run_in_batches(handler, img_root, mask_root, pairs, batch_size=1, num_workers=4, save_path="/content/results"):
    assert batch_size == 1, "Batch size must be 1"
    os.makedirs(save_path, exist_ok=True)

    dataset = FaceSwapDataset(img_root, mask_root, pairs)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    for batch in tqdm(dataloader):
        process_batch(batch, handler, save_path)

        gc.collect()
        torch.cuda.empty_cache()


In [ ]:
%cd One-Shot-Face-Swapping-on-Megapixels/inference

[Errno 2] No such file or directory: 'One-Shot-Face-Swapping-on-Megapixels/inference'
/content/One-Shot-Face-Swapping-on-Megapixels/inference


In [ ]:
import random


img_root = "/content/CelebAMask-HQ/CelebA-HQ-img"
mask_root = "/content/content/masks_output"

handler = My_MegaFS(swap_type="ftm",
                    img_root=img_root,
                    mask_root=mask_root)

# # Generate all pairs (example: pairing images with themselves)
pairs = [(str(i), str(i)) for i in range(30000)]
rd1 = random.sample(range(0, 30000), 10000)
rd2 = random.sample(range(0, 30000), 10000)

pairs = [(str(i), str(j)) for i, j in zip(rd1, rd2)]


load encoder & swapper: ./checkpoint/ftm_final.pth


/content/One-Shot-Face-Swapping-on-Megapixels/inference/inference.py:79: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpts = torch.load(ckpt_e, map_location=torch.device("

load generator: ./checkpoint/stylegan2-ffhq-config-f.pth


/content/One-Shot-Face-Swapping-on-Megapixels/inference/inference.py:88: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpts = torch.load(ckpt_f, map_location=torch.device("

In [ ]:
run_in_batches(handler, img_root, mask_root, pairs, batch_size=1, num_workers=4)


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


  0%|          | 0/10000 [00:00<?, ?it/s]

/content/One-Shot-Face-Swapping-on-Megapixels/inference/megafs.py:270: UserWarning: `nn.functional.upsample` is deprecated. Use `nn.functional.interpolate` instead.
  p2 = self.output2(c2) + F.upsample(p3, scale_factor=2, mode='bilinear', align_corners=True) # N, 512, 16, 16
/content/One-Shot-Face-Swapping-on-Megapixels/inference/megafs.py:271: UserWarning: `nn.functional.upsample` is deprecated. Use `nn.functional.interpolate` instead.
  p1 = self.output1(c1) + F.upsample(p2, scale_factor=2, mode='bilinear', align_corners=True) # N, 512, 32, 32


In [ ]:
import shutil


shutil.move("/content/generated_imgs.zip", "/content/train")

In [ ]:
!zip -r /content/generated_imgs.zip /content/results

Выходные данные были обрезаны до нескольких последних строк (5000).
  adding: content/results/ftm_src_7016__tgt_28369.jpg (deflated 0%)
  adding: content/results/ftm_src_20464__tgt_04837.jpg (deflated 0%)
  adding: content/results/ftm_src_18284__tgt_28103.jpg (deflated 0%)
  adding: content/results/ftm_src_967__tgt_22330.jpg (deflated 0%)
  adding: content/results/ftm_src_15357__tgt_19587.jpg (deflated 0%)
  adding: content/results/ftm_src_5130__tgt_29465.jpg (deflated 0%)
  adding: content/results/ftm_src_9629__tgt_23904.jpg (deflated 0%)
  adding: content/results/ftm_src_18629__tgt_29679.jpg (deflated 0%)
  adding: content/results/ftm_src_18831__tgt_09525.jpg (deflated 0%)
  adding: content/results/ftm_src_22297__tgt_03464.jpg (deflated 0%)
  adding: content/results/ftm_src_6010__tgt_00491.jpg (deflated 0%)
  adding: content/results/ftm_src_15117__tgt_02724.jpg (deflated 0%)
  adding: content/results/ftm_src_9992__tgt_21340.jpg (deflated 0%)
  adding: content/results/ftm_src_2069__tg